# Q Learning



In Q Learning we try to learn the Expected Future Discounted Reward for each possible action.
We then try to maximize our score function by taking the action with the best Q Value.

The Q Function is given by:
$
\begin{align}
Q^{\pi}(s_{t}, a_{t}) = E[R_{t+1} + R_{t+2} + R_{t+3} .... | s_{t}, a_{t}]
\end{align}
$

We optimize it using the following equation:

$
\begin{align}
Q(s, a) = Q(s, a) + \alpha [r + \gamma max_{a^{'}}Q(s^{'}, a^{'}) - Q(s, a)]
\end{align}
$


$
\begin{align}
\text{Where } \gamma   \text{: is the discount factor}
\end{align}
$

The intuition behind this is that if we are we move from a good state to a state with all bad future estimates, the current Q value would decrease and when we move to a state that has a better Q value then we would increase the current Q Value.

We do not actually have to wait until the end of the episode to calculate the future discounted reward. Instead we use the estimated future discounted reward by looking up our Q Table and taking the maximum value over all posible actions.

This approach is called a Temporal Difference Method.





In [2]:
from IPython.display import clear_output
import time
import gym
import numpy as np
import random

In [3]:
# Create Environment and Render Sample
env = gym.make("Taxi-v2").env
env.render()

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



In [5]:
# View Action Space and Observation Space
state = env.reset()
env.render()

print(f"Action Space: {env.action_space}")
print(f"State Space: {env.observation_space}")

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

Action Space:  Discrete(6)
State Space:  Discrete(500)


### Visualize Random Agent

In [7]:
steps = 0
penalties, reward = 0, 0

frames = []

done = False

# Simple Loop with the agent taking random samples. 
# This will take some time as it takes alot of time steps to ramdomly drop the passenger to the destination.

while not done:
    clear_output(wait = True) # Used to clear the IPython display for the cell. 
    action = env.action_space.sample()
    env.render()
    state, reward, done, info = env.step(action)

    if reward == -10:
        penalties += 1
    
    steps += 1
    time.sleep(.1) # Sleep so that we can visualize each state.
    
    
print(f"Timesteps taken: {steps}")
print(f"Penalties incurred: {penalties}")

+---------+
|R: | : :G|
| : : : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
Timesteps taken:  4271
Penalties incurred:  1352


### Create Q Table to Store Q value for each State Action pair

In [9]:
# The Q table contains the Q Value for each State-Action pair. So create a table with states x actions.
q_table = np.zeros([env.observation_space.n, env.action_space.n])

### Train Agent

In [10]:
alpha = 0.1
gamma = 0.6
epsilon = 0.1

all_epochs = []
all_penalties = []

for i in range(1, 100001):
    state = env.reset()

    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    while not done:
        # Take a random action if sampled random number < epsilon. This promotes exploring.
        # Otherwise take best action from Q Table for that state.
        if random.uniform(0, 1) < epsilon:
            action = env.action_space.sample() 
        else:
            action = np.argmax(q_table[state])

        next_state, reward, done, info = env.step(action) 
        
        old_value = q_table[state, action]
        next_max = np.max(q_table[next_state])
        
        if reward == -10:
            penalties += 1
        
        # Update the Q table using the equation above.
        new_value = old_value + alpha * (reward + gamma * next_max - old_value)
        q_table[state, action] = new_value

        state = next_state
        epochs += 1
    
    if i % 100 == 0:
        clear_output(wait=True)
        print(f"Episode: {i}")

Episode: 100000


### Evaluate agents performance

In [11]:
# Evaluate the agent across 100 epochs by always taking the best action.
total_steps, total_penalties = 0, 0
episodes = 100

for _ in range(episodes):
    state = env.reset()
    steps, penalties, reward = 0, 0, 0
    
    done = False
    
    while not done:
        action = np.argmax(q_table[state])
        state, reward, done, info = env.step(action)

        if reward == -10:
            penalties += 1

        steps += 1
    
    total_penalties += penalties
    total_steps += steps

print(f"Results after {episodes} episodes:")
print(f"Average timesteps per episode: {total_steps / episodes}")
print(f"Average penalties per episode: {total_penalties / episodes}")

Results after 100 episodes:
Average timesteps per episode: 12.87
Average penalties per episode: 0.0
